# Kasus 3: Kapasitas Kontainer dan Informasi Pelabuhan

---

## Entity Relationship Diagram (ERD)


In [1]:
import base64
from IPython.display import Image, display

# credits: https://gist.github.com/MLKrisJohnson/2d2df47879ee6afd3be9d6788241fe99


def mm_ink(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, return a URL that will generate the image."""
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    return "https://mermaid.ink/img/" + base64_string


def mm_display(graphbytes):
    """Given a bytes object holding a Mermaid-format graph, display it."""
    display(Image(url=mm_ink(graphbytes)))


def mm(graph):
    """Given a string containing a Mermaid-format graph, display it."""
    graphbytes = graph.encode("ascii")
    mm_display(graphbytes)


def mm_link(graph):
    """Given a string containing a Mermaid-format graph, return URL for display."""
    graphbytes = graph.encode("ascii")
    return mm_ink(graphbytes)


def mm_path(path):
    """Given a path to a file containing a Mermaid-format graph, display it"""
    with open(path, 'rb') as f:
        graphbytes = f.read()
    mm_display(graphbytes)


mm("""
erDiagram
    port_details ||--|{ port_container_capacity : consists_of
    port_details {
        INT port_id PK
        VARCHAR port_name
        VARCHAR location
    }
    port_container_capacity {
        INT id PK
        INT port_id FK
        INT year
        INT month
        INT capacity
    }
""")

## Load Sample Data into an SQLite Database


In [2]:
import pandas as pd
from sqlite3 import connect
from IPython.display import HTML

# Helper class to display tables side-by-side
# credits: https://python.plainenglish.io/displaying-multiple-dataframes-side-by-side-in-jupyter-lab-notebook-9a4649a4940


def side_by_side(*dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 2em">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))


# Load data from CSV files
port_details_file_path = './3-port_details.csv'
port_capacity_file_path = './3-port_container_capacity.csv'
details_data = pd.read_csv(port_details_file_path)
capacity_data = pd.read_csv(port_capacity_file_path)

# Create an in-memory SQLite database
conn = connect(':memory:')
details_data.to_sql(name='details', con=conn)
capacity_data.to_sql(name='capacities', con=conn)

side_by_side(details_data.head(), capacity_data.head())

# Task 1

> 1. Tulis query untuk menghasilkan nama lokasi, kuartal (Q1, Q2, Q3, Q4), dan total kapasitas
>    kontainer.


In [3]:
pd.read_sql('''SELECT d.location, c.year,
            CASE
              WHEN c.month >= 1 AND c.month <= 3 THEN 1
              WHEN c.month >= 4 AND c.month <= 6 THEN 2
              WHEN c.month >= 7 AND c.month <= 9 THEN 3
              WHEN c.month >= 10 AND c.month <= 12 THEN 4
            END AS quarter,
            SUM(c.capacity) as total_capacity
            FROM details d
            LEFT JOIN capacities c ON c.port_id = d.port_id
            GROUP BY d.location, c.year, quarter
            ORDER BY d.location, c.year, c.month''', conn)

,location,year,quarter,total_capacity
0,Jakarta,2022,1,870000
1,Jakarta,2022,2,1020000
2,Jakarta,2022,3,1030000
3,Jakarta,2022,4,950000
4,Jakarta,2023,1,930000
5,Jakarta,2023,2,1060000
6,Jakarta,2023,3,1050000
7,Jakarta,2023,4,950000
8,North Sumatra,2022,1,310000
9,North Sumatra,2022,2,330000


# Task 2

> 2. Tulis query untuk mendapatkan lokasi, rata-rata kapasitas tahunan, dan perbedaan rata-rata kapasitas antara kedua lokasi.

Interpretasi:

- `rata-rata kapasitas tahunan` adalah rata-rata kapasitas di suatu tahun untuk seluruh lokasi, menghasilkan satu value.
- `perbedaan rata-rata kapasitas antara kedua lokasi` adalah selisih antara rata-rata tahunan suatu lokasi dengan rata-rata tahunan untuk seluruh lokasi.


In [4]:
pd.read_sql('''SELECT data.*,
                AVG(data.location_avg_capacity) OVER (PARTITION BY data.year)
                  as national_avg_capacity,
                (data.location_avg_capacity - AVG(data.location_avg_capacity) OVER (PARTITION BY data.year))
                  as difference_from_national_avg
            FROM (
              SELECT c.year as year,
                d.location as location,
                AVG(c.capacity) as location_avg_capacity
              FROM details d LEFT JOIN capacities c ON c.port_id = d.port_id
              GROUP BY c.year, d.location
              ORDER BY c.year, d.location
            ) data
            ORDER BY data.year, data.location
            ''', conn)

,year,location,location_avg_capacity,national_avg_capacity,difference_from_national_avg
0,2022,Jakarta,322500.000000,216388.750000,106111.250000
1,2022,North Sumatra,110277.500000,216388.750000,-106111.250000
2,2023,Jakarta,332500.000000,223194.458333,109305.541667
3,2023,North Sumatra,113888.916667,223194.458333,-109305.541667


# Task 3

> 3. Tulis query untuk mendapatkan nama pelabuhan, lokasi, dan bulan-bulan di mana kapasitas meningkat dibandingkan bulan sebelumnya.


In [5]:
pd.read_sql('''SELECT data.*, (data.capacity - data.prev_capacity) as capacity_growth
            FROM (
              SELECT d.port_name as port_name,
                d.location as location,
                c.year as year,
                c.month as month,
                c.capacity as capacity,
                LAG (c.capacity, 1, 0) OVER (PARTITION BY d.port_id) as prev_capacity
              FROM details d LEFT JOIN capacities c ON c.port_id = d.port_id
            ) data
            WHERE capacity_growth > 0
            ORDER BY data.port_name, data.year, data.month''', conn)

,port_name,location,year,month,capacity,prev_capacity,capacity_growth
0,Belawan,North Sumatra,2022,1,85000,0,85000
1,Belawan,North Sumatra,2022,2,105000,85000,20000
2,Belawan,North Sumatra,2022,3,120000,105000,15000
3,Belawan,North Sumatra,2022,5,125000,95000,30000
4,Belawan,North Sumatra,2022,7,115000,110000,5000
5,Belawan,North Sumatra,2022,9,130000,100000,30000
6,Belawan,North Sumatra,2022,11,118330,90000,28330
7,Belawan,North Sumatra,2022,12,130000,118330,11670
8,Belawan,North Sumatra,2023,2,108000,88000,20000
9,Belawan,North Sumatra,2023,3,125000,108000,17000
